In [1]:
import pandas as pd 
import json 

import os 
import sys

import pdb 

In [2]:
names = json.load(open("../../data/names_top_2.json"))
verbs = ["promised"]
actions = json.load(open("../../data/verbs.json"))
correct_index = 0

nicknames = json.load(open("../../data/nicknames.json"))


In [3]:
# 
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import re 

class HuggingfaceRunFxn:
    def __init__(self, model_name, constrained = False, device="cpu"): 
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.tokenizer.padding_side = "left" 
        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = GPTNeoForCausalLM.from_pretrained(model_name)
        self.device = device 
        self.model.to(device)
        print(self.model.config.__dict__)
        self.model.config.return_dict_in_generate=True
        self.constrained = constrained

    def get_names(self, text): 
        # get names from the prompt 
        n1, n2 = re.findall("Answer the question with either \"(\w+)\" or \"(\w+)\"", text)[0]
        return n1, n2

    def __call__(self, text, kwargs):
        input_ids = self.tokenizer.encode(text, return_tensors='pt')
        input_ids = input_ids.to(self.device)
        outputs =  self.model.generate(input_ids, 
                                       output_scores = True, 
                                       return_dict_in_generate=True, 
                                       num_beams=1, 
                                       max_length=100)
        print(outputs)
        if not self.constrained:
            output_text = self.tokenizer.decode(outputs[0].to("cpu"), skip_special_tokens=True)
        else:
            n1, n2 = self.get_names(text)
            output_text = self.tokenizer.decode(outputs[0].to("cpu"), skip_special_tokens=True)
            # print(type(outputs))
            # print(outputs)

        return output_text 

In [4]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/brtx/601-nvme1/estengel/.cache"



prompt = """You will be given a context and a question. Answer the question with either "Avery" or "Casey".
Context: Avery told Casey to come.

Question:  Who came, Avery or Casey?
Answer: """



fxn = HuggingfaceRunFxn("EleutherAI/gpt-neo-1.3B", constrained=True, device="cpu")

res = fxn(prompt, None)
print(res)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

OSError: Unable to load weights from pytorch checkpoint file for 'EleutherAI/gpt-neo-1.3B' at '/brtx/601-nvme1/estengel/.cache/7c5fac9d60b015cbc7c007ab8fe6d0512787fbaef81968922959898c49468d73.4c6a483fbfb5a25ac384bfcd71a1ff15245f06583a00c4ab4c44ed0f761f0b08'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

3.0.2
